In [124]:
# Ô 1: Import thư viện và thiết lập đường dẫn 
import os
import sys
import random
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from tqdm import tqdm 

PROJECT_ROOT_DIR = r"D:\code_etc\Python\_File_code\Pose_estimation_Final"
# --------------------------------

if PROJECT_ROOT_DIR not in sys.path:
    sys.path.insert(0, PROJECT_ROOT_DIR) 

print(f"Đã thêm vào sys.path: {PROJECT_ROOT_DIR}")

try:
    from src.models.stgcn import Model as STGCN
    print("Import STGCN thành công!")
except ImportError as e:
    print(f"LỖI: Không thể import STGCN từ src.models.stgcn: {e}")
    print("Vui lòng kiểm tra lại đường dẫn tuyệt đối PROJECT_ROOT_DIR và cấu trúc thư mục.")
    # Dừng thực thi ô này nếu import lỗi
    raise

# --- ĐỊNH NGHĨA HÀM get_random_skeleton_file ---
def get_random_skeleton_file(base_dir):
    try:
        action_classes = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]
        if not action_classes: print(f"Lỗi: Không tìm thấy thư mục lớp trong '{base_dir}'"); return None
        random_class = random.choice(action_classes)
        class_dir = os.path.join(base_dir, random_class)
        print(f"Đã chọn lớp ngẫu nhiên: {random_class}")
        skeleton_files = [f for f in os.listdir(class_dir) if f.endswith('.skeleton')]
        if not skeleton_files: print(f"Lỗi: Không tìm thấy file .skeleton trong '{class_dir}'"); return None
        random_file_name = random.choice(skeleton_files)
        full_path = os.path.join(class_dir, random_file_name)
        return full_path
    except FileNotFoundError: print(f"Lỗi: Không tìm thấy thư mục '{base_dir}'"); return None
    except Exception as e: print(f"Lỗi không mong muốn: {e}"); return None
# --- KẾT THÚC HÀM ---

# --- CẤU HÌNH ---
MODEL_TYPE = 'best'

if MODEL_TYPE == 'best':
    WEIGHTS_PATH = os.path.join(PROJECT_ROOT_DIR, 'weights', 'best_finetuned_model.pt')
else:
    WEIGHTS_PATH = os.path.join(PROJECT_ROOT_DIR, 'weights', 'last_epoch_model.pt')

CUSTOM_SKELETONS_DIR_PATH = os.path.join(PROJECT_ROOT_DIR, 'data', 'processed', 'custom_skeletons')
print(f"Đang tìm file skeleton ngẫu nhiên trong: {CUSTOM_SKELETONS_DIR_PATH}")
SAMPLE_SKELETON_FILE = get_random_skeleton_file(CUSTOM_SKELETONS_DIR_PATH)
if SAMPLE_SKELETON_FILE is None:
    print("LỖI: Không tìm được file skeleton ngẫu nhiên. Dùng file NTU thay thế.")
    SAMPLE_SKELETON_FILE = os.path.join(PROJECT_ROOT_DIR, 'data', 'processed', 'ntu_filtered_skeletons', 'running', 'S001C001P002R002A007.skeleton')

NUM_CLASSES = 6
CLASS_NAMES = ['walking', 'running', 'jumping', 'standing_up', 'carrying', 'lying_down']
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f"\nThư mục gốc dự án: {PROJECT_ROOT_DIR}")
print(f"Đang sử dụng thiết bị: {DEVICE}")
print(f"Đang test model: {WEIGHTS_PATH}")
print(f"Đang test file: {SAMPLE_SKELETON_FILE}")

# --- SAO CHÉP HÀM parse_skeleton_file ---
def parse_skeleton_file(filepath):
    # ... (Nội dung hàm parse_skeleton_file giữ nguyên như bạn đã có) ...
    try:
        with open(filepath, 'r') as f: lines = f.readlines()
        if not lines: tqdm.write(f"\n[Lỗi Parse] File {filepath} trống."); return None
        frame_count = int(lines[0].strip())
        frames_data = []; line_idx = 1; valid_frames_count = 0
        for i in range(frame_count):
            if line_idx + 1 >= len(lines): tqdm.write(f"\n[Cảnh báo Parse] File {filepath}: Thiếu dòng body count ở frame {i}. Dừng đọc file."); break
            try:
                body_count = int(lines[line_idx].strip()); line_idx += 1
                if body_count == 0:
                     temp_idx = line_idx; found_next_frame = False
                     while temp_idx < len(lines):
                          try: next_body_count = int(lines[temp_idx].strip()); line_idx = temp_idx; found_next_frame = True; break
                          except ValueError: temp_idx += 1
                     if not found_next_frame: line_idx = len(lines)
                     continue
                best_body_joints = None
                for j in range(body_count):
                    if line_idx + 1 >= len(lines): tqdm.write(f"\n[Cảnh báo Parse] File {filepath}: Thiếu dòng body info/joint count ở frame {i}, body {j}. Dừng đọc file."); line_idx = len(lines); break
                    try: line_idx += 1; joint_count = int(lines[line_idx].strip()); line_idx += 1
                    except (ValueError, IndexError): tqdm.write(f"\n[Cảnh báo Parse] File {filepath}: Lỗi đọc body info/joint count ở frame {i}, body {j}. Bỏ qua body."); line_idx += 25; continue
                    if line_idx + joint_count > len(lines): tqdm.write(f"\n[Cảnh báo Parse] File {filepath}: Thiếu dòng khớp ở frame {i}, body {j} (cần {joint_count}, còn {len(lines)-line_idx}). Dừng đọc file."); line_idx = len(lines); break
                    current_body_joints = []; joint_lines_read = 0
                    for k in range(joint_count):
                        try: joint_info = lines[line_idx].strip().split(); current_body_joints.append([float(coord) for coord in joint_info[:3]]); line_idx += 1; joint_lines_read += 1
                        except (ValueError, IndexError): tqdm.write(f"\n[Cảnh báo Parse] File {filepath}: Lỗi đọc khớp {k} ở frame {i}, body {j}. Bỏ qua body."); line_idx += (joint_count - joint_lines_read); current_body_joints = None; break
                    if j == 0 and current_body_joints is not None:
                        while len(current_body_joints) < 25: current_body_joints.append([0.0, 0.0, 0.0])
                        best_body_joints = np.array(current_body_joints[:25])
                if line_idx >= len(lines): break
                if best_body_joints is not None: frames_data.append(best_body_joints); valid_frames_count += 1
            except (ValueError, IndexError) as e_frame:
                 tqdm.write(f"\n[Cảnh báo Parse] File {filepath}: Lỗi đọc body count ở frame {i}: {e_frame}. Thử bỏ qua frame.")
                 temp_idx = line_idx + 1; found_next_frame = False
                 while temp_idx < len(lines):
                      try: next_body_count = int(lines[temp_idx].strip()); line_idx = temp_idx; found_next_frame = True; break
                      except ValueError: temp_idx += 1
                 if not found_next_frame: line_idx = len(lines)
                 if line_idx >= len(lines): break
                 continue
        if valid_frames_count > 0: return np.array(frames_data)
        else: tqdm.write(f"\n[Lỗi Dataset] File {filepath} không đọc được frame hợp lệ nào sau khi xử lý."); return None
    except Exception as e:
        tqdm.write(f"\n[Lỗi Dataset] Lỗi nghiêm trọng khi đọc file {filepath}: {e}")
        return None
# --- KẾT THÚC HÀM parse ---

# --- SAO CHÉP HÀM plot_skeleton ---
def plot_skeleton(joints_data, frame_num):
    # ... (Nội dung hàm plot_skeleton giữ nguyên) ...
    x = joints_data[:, 0]; y = joints_data[:, 1]
    connections = [(0, 1), (1, 20), (20, 2), (2, 3), (20, 4), (4, 5), (5, 6), (6, 7), (7, 21), (7, 22), (20, 8), (8, 9), (9, 10), (10, 11), (11, 23), (11, 24), (0, 12), (12, 13), (13, 14), (14, 15), (0, 16), (16, 17), (17, 18), (18, 19)]
    plt.figure(figsize=(6, 8)); plt.scatter(x, y, c='red', s=40)
    for (start_joint, end_joint) in connections:
        if start_joint < len(x) and end_joint < len(x): plt.plot([x[start_joint], x[end_joint]], [y[start_joint], y[end_joint]], 'b-')
    plt.title(f"Khung xương 2D - Frame {frame_num}"); plt.xlabel("Tọa độ X"); plt.ylabel("Tọa độ Y"); plt.gca().set_aspect('equal', adjustable='box'); plt.show()
# --- KẾT THÚC HÀM plot ---

Đã thêm vào sys.path: D:\code_etc\Python\_File_code\Pose_estimation_Final
Import STGCN thành công!
Đang tìm file skeleton ngẫu nhiên trong: D:\code_etc\Python\_File_code\Pose_estimation_Final\data\processed\custom_skeletons
Đã chọn lớp ngẫu nhiên: running

Thư mục gốc dự án: D:\code_etc\Python\_File_code\Pose_estimation_Final
Đang sử dụng thiết bị: cuda
Đang test model: D:\code_etc\Python\_File_code\Pose_estimation_Final\weights\best_finetuned_model.pt
Đang test file: D:\code_etc\Python\_File_code\Pose_estimation_Final\data\processed\custom_skeletons\running\video_002_custom.skeleton


In [125]:
# Ô 2: Khởi tạo Model và Tải Trọng số Fine-tuned

# Khởi tạo kiến trúc model với đúng SỐ LỚP ĐẦU RA (NUM_CLASSES)
graph_args = {'layout': 'ntu-rgb+d', 'strategy': 'spatial'}
model = STGCN(in_channels=3, num_class=NUM_CLASSES, graph_args=graph_args, edge_importance_weighting=True)

# Tải trọng số đã fine-tune
print("Đang tải trọng số fine-tuned...")
try:
    model.load_state_dict(torch.load(WEIGHTS_PATH, weights_only=True, map_location=DEVICE)) # Thêm map_location
    model.to(DEVICE)
    model.eval() # Chuyển sang chế độ đánh giá
    print("Tải model thành công!")
except FileNotFoundError:
    print(f"LỖI: Không tìm thấy file trọng số tại {WEIGHTS_PATH}")
except Exception as e:
    print(f"LỖI khi tải model: {e}")

Đang tải trọng số fine-tuned...
Tải model thành công!


In [126]:
# Ô 3: Tải và Chuẩn bị Dữ liệu Mẫu

# Đọc file skeleton mẫu
skeleton_data_raw = parse_skeleton_file(SAMPLE_SKELETON_FILE) # Shape (T, 25, 3)

if skeleton_data_raw is None or skeleton_data_raw.shape[0] == 0:
    print("Lỗi: Không thể đọc hoặc file skeleton rỗng.")
else:
    print(f"Đã đọc file skeleton. Số frame: {skeleton_data_raw.shape[0]}, Số khớp: {skeleton_data_raw.shape[1]}")

    # --- Tiền xử lý giống hệt trong SkeletonDataset ---
    T, V, C = skeleton_data_raw.shape
    padded_data = np.zeros((MAX_FRAMES, V, C), dtype=np.float32)
    if T >= MAX_FRAMES:
        indices = np.linspace(0, T - 1, MAX_FRAMES, dtype=int)
        padded_data = skeleton_data_raw[indices]
    else:
        padded_data[:T] = skeleton_data_raw

    # Chuẩn bị cho ST-GCN: (T, V, C) -> (N=1, C, T, V, M=1)
    data_final = np.transpose(padded_data, (2, 0, 1))
    data_final = np.expand_dims(data_final, axis=-1) # Thêm chiều M
    data_final = np.expand_dims(data_final, axis=0)  # Thêm chiều N (batch size = 1)

    # Chuyển thành Tensor
    input_tensor = torch.FloatTensor(data_final).to(DEVICE)
    print(f"Đã chuẩn bị xong input tensor với shape: {input_tensor.shape}")

Đã đọc file skeleton. Số frame: 52, Số khớp: 25
Đã chuẩn bị xong input tensor với shape: torch.Size([1, 3, 300, 25, 1])


In [127]:
# Ô 4: Chạy Dự đoán (Inference)

if 'input_tensor' in locals(): # Kiểm tra xem tensor đã được tạo chưa
    print("\nĐang chạy dự đoán...")
    with torch.no_grad():
        output = model(input_tensor) # Output shape: (1, 1, 6) or (1, 6)

    # Xử lý output
    # Squeeze các chiều không cần thiết (N và M)
    output_squeezed = output.squeeze() # Shape (6)

    # Tính xác suất bằng Softmax
    probabilities = torch.nn.functional.softmax(output_squeezed, dim=0)

    # Lấy lớp có xác suất cao nhất
    confidence, predicted_index = torch.max(probabilities, 0)
    predicted_class_name = CLASS_NAMES[predicted_index.item()]

    print("\n--- KẾT QUẢ DỰ ĐOÁN ---")
    print(f"Hành động: {predicted_class_name}")
    print(f"Độ tin cậy: {confidence.item():.4f}")

    # In ra xác suất của tất cả các lớp (tùy chọn)
    print("\nXác suất các lớp:")
    for i, prob in enumerate(probabilities):
        print(f"- {CLASS_NAMES[i]}: {prob.item():.4f}")

else:
    print("\nKhông có input tensor để chạy dự đoán do lỗi đọc file.")


Đang chạy dự đoán...

--- KẾT QUẢ DỰ ĐOÁN ---
Hành động: running
Độ tin cậy: 0.9575

Xác suất các lớp:
- walking: 0.0096
- running: 0.9575
- jumping: 0.0081
- standing_up: 0.0178
- carrying: 0.0029
- lying_down: 0.0041
